In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [152]:
#計測地点のosm_idを指定
start_osm = 266396723
mid_osm = [232919754,651358177]
end_osm = 637167088
start_mesh = 50325662142
end_mesh = 50325652441

In [153]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [154]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun_4d_30"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [155]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.97927765084002距離0.14468426940260257 km
存在メッシュ：50325662142値31.410714285714285距離0.0 km
存在メッシュ：50325662213値53.8938428240289距離0.1851577137438622 km
存在メッシュ：50325662231値48.97927765084002距離0.14468426940260257 km
存在メッシュ：50325662142値31.410714285714285距離0.0 km
存在メッシュ：50325662232値66.54784101528375距離0.2893685387995886 km
存在メッシュ：50325662124値45.44042955923615距離0.11554041512042086 km
存在メッシュ：50325662213値53.8938428240289距離0.1851577137438622 km
存在メッシュ：50325662231値48.97927765084002距離0.14468426940260257 km
存在メッシュ：50325662232値66.54784101528375距離0.2893685387995886 km
存在メッシュ：50325662213値53.8938428240289距離0.1851577137438622 km
存在メッシュ：50325662142値31.410714285714285距離0.0 km
存在メッシュ：50325662232値66.54784101528375距離0.2893685387995886 km
存在メッシュ：50325662231値48.97927765084002距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値50.37332046823886距離0.14468426940260257 km
存在メッシュ：50325662142値33.69491

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値25.63080500312425距離0.14468426940260257 km
存在メッシュ：50325662142値16.883333333333333距離0.0 km
存在メッシュ：50325662213値28.077790492395827距離0.1851577137438622 km
存在メッシュ：50325662231値25.63080500312425距離0.14468426940260257 km
存在メッシュ：50325662142値16.883333333333333距離0.0 km
存在メッシュ：50325662232値34.378276672575595距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値28.077790492395827距離0.1851577137438622 km
存在メッシュ：50325662231値25.63080500312425距離0.14468426940260257 km
存在メッシュ：50325662232値34.378276672575595距離0.2893685387995886 km
存在メッシュ：50325662213値28.077790492395827距離0.1851577137438622 km
存在メッシュ：50325662142値16.883333333333333距離0.0 km
存在メッシュ：50325662232値34.378276672575595距離0.2893685387995886 km
存在メッシュ：50325662231値25.63080500312425距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値22.48939116947849距離0.14468426940260257 km
存在メッシュ：50325662142値14.616666666666667距離0.0 km
存在メッシュ：

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値9.588677092312258距離0.14468426940260257 km
存在メッシュ：50325662142値6.766666666666667距離0.0 km
存在メッシュ：50325662213値10.378095889285959距離0.1851577137438622 km
存在メッシュ：50325662231値9.588677092312258距離0.14468426940260257 km
存在メッシュ：50325662142値6.766666666666667距離0.0 km
存在メッシュ：50325662232値12.410687517848302距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値10.378095889285959距離0.1851577137438622 km
存在メッシュ：50325662231値9.588677092312258距離0.14468426940260257 km
存在メッシュ：50325662232値12.410687517848302距離0.2893685387995886 km
存在メッシュ：50325662213値10.378095889285959距離0.1851577137438622 km
存在メッシュ：50325662142値6.766666666666667距離0.0 km
存在メッシュ：50325662232値12.410687517848302距離0.2893685387995886 km
存在メッシュ：50325662231値9.588677092312258距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値8.371114522010178距離0.14468426940260257 km
存在メッシュ：50325662142値5.583333333333333距離0.0 km
存在メッシュ：5032

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値39.277816964946766距離0.14468426940260257 km
存在メッシュ：50325662142値25.833333333333332距離0.0 km
存在メッシュ：50325662213値43.03872727128373距離0.1851577137438622 km
存在メッシュ：50325662231値39.277816964946766距離0.14468426940260257 km
存在メッシュ：50325662142値25.833333333333332距離0.0 km
存在メッシュ：50325662232値52.722300596038295距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値43.03872727128373距離0.1851577137438622 km
存在メッシュ：50325662231値39.277816964946766距離0.14468426940260257 km
存在メッシュ：50325662232値52.722300596038295距離0.2893685387995886 km
存在メッシュ：50325662213値43.03872727128373距離0.1851577137438622 km
存在メッシュ：50325662142値25.833333333333332距離0.0 km
存在メッシュ：50325662232値52.722300596038295距離0.2893685387995886 km
存在メッシュ：50325662231値39.277816964946766距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値32.46713251934276距離0.14468426940260257 km
存在メッシュ：50325662142値20.916666666666668距離0.0 km
存在メッシュ

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値15.797468597437073距離0.14468426940260257 km
存在メッシュ：50325662142値10.8距離0.0 km
存在メッシュ：50325662213値17.19544204652527距離0.1851577137438622 km
存在メッシュ：50325662231値15.797468597437073距離0.14468426940260257 km
存在メッシュ：50325662142値10.8距離0.0 km
存在メッシュ：50325662232値20.79493719468015距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値17.19544204652527距離0.1851577137438622 km
存在メッシュ：50325662231値15.797468597437073距離0.14468426940260257 km
存在メッシュ：50325662232値20.79493719468015距離0.2893685387995886 km
存在メッシュ：50325662213値17.19544204652527距離0.1851577137438622 km
存在メッシュ：50325662142値10.8距離0.0 km
存在メッシュ：50325662232値20.79493719468015距離0.2893685387995886 km
存在メッシュ：50325662231値15.797468597437073距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値13.758303206676699距離0.14468426940260257 km
存在メッシュ：50325662142値9.266666666666667距離0.0 km
存在メッシュ：50325662213値15.014777060080931距離0.1851577137

...

存在メッシュ：50325662231値6.309805909562293距離0.14468426940260257 km
存在メッシュ：50325662142値3.8833333333333333距離0.0 km
存在メッシュ：50325662213値6.988578406151539距離0.1851577137438622 km
存在メッシュ：50325662231値6.309805909562293距離0.14468426940260257 km
存在メッシュ：50325662142値3.8833333333333333距離0.0 km
存在メッシュ：50325662232値8.736278485697058距離0.2893685387995886 km
存在メッシュ：50325662124値5.82103982964694距離0.11554041512042086 km
存在メッシュ：50325662213値6.988578406151539距離0.1851577137438622 km
存在メッシュ：50325662231値6.309805909562293距離0.14468426940260257 km
存在メッシュ：50325662232値8.736278485697058距離0.2893685387995886 km
存在メッシュ：50325662213値6.988578406151539距離0.1851577137438622 km
存在メッシュ：50325662142値3.8833333333333333距離0.0 km
存在メッシュ：50325662232値8.736278485697058距離0.2893685387995886 km
存在メッシュ：50325662231値6.309805909562293距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.97927765084002距離0.14468426940260257 km
存在メッシュ：50325662142値31.410714285714285距離0.0 km
存在メッシュ：50325662213値53.8938428240289

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値26.90487590047836距離0.14468426940260257 km
存在メッシュ：50325662142値17.45距離0.0 km
存在メッシュ：50325662213値29.54974804236059距離0.1851577137438622 km
存在メッシュ：50325662231値26.90487590047836距離0.14468426940260257 km
存在メッシュ：50325662142値17.45距離0.0 km
存在メッシュ：50325662232値36.359751800589684距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値29.54974804236059距離0.1851577137438622 km
存在メッシュ：50325662231値26.90487590047836距離0.14468426940260257 km
存在メッシュ：50325662232値36.359751800589684距離0.2893685387995886 km
存在メッシュ：50325662213値29.54974804236059距離0.1851577137438622 km
存在メッシュ：50325662142値17.45距離0.0 km
存在メッシュ：50325662232値36.359751800589684距離0.2893685387995886 km
存在メッシュ：50325662231値26.90487590047836距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値25.63080500312425距離0.14468426940260257 km
存在メッシュ：50325662142値16.883333333333333距離0.0 km
存在メッシュ：50325662213値28.077790492395827距離0.18515771

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値11.02648994382266距離0.14468426940260257 km
存在メッシュ：50325662142値8.033333333333333距離0.0 km
存在メッシュ：50325662213値11.863784543847325距離0.1851577137438622 km
存在メッシュ：50325662231値11.02648994382266距離0.14468426940260257 km
存在メッシュ：50325662142値8.033333333333333距離0.0 km
存在メッシュ：50325662232値14.019646554195795距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値11.863784543847325距離0.1851577137438622 km
存在メッシュ：50325662231値11.02648994382266距離0.14468426940260257 km
存在メッシュ：50325662232値14.019646554195795距離0.2893685387995886 km
存在メッシュ：50325662213値11.863784543847325距離0.1851577137438622 km
存在メッシュ：50325662142値8.033333333333333距離0.0 km
存在メッシュ：50325662232値14.019646554195795距離0.2893685387995886 km
存在メッシュ：50325662231値11.02648994382266距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値9.588677092312258距離0.14468426940260257 km
存在メッシュ：50325662142値6.766666666666667距離0.0 km
存在メッシュ：5032

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値38.765713582071825距離0.14468426940260257 km
存在メッシュ：50325662142値26.066666666666666距離0.0 km
存在メッシュ：50325662213値42.31809816845348距離0.1851577137438622 km
存在メッシュ：50325662231値38.765713582071825距離0.14468426940260257 km
存在メッシュ：50325662142値26.066666666666666距離0.0 km
存在メッシュ：50325662232値51.46476049698403距離0.2893685387995886 km
存在メッシュ：50325662124値36.20773561837691距離0.11554041512042086 km
存在メッシュ：50325662213値42.31809816845348距離0.1851577137438622 km
存在メッシュ：50325662231値38.765713582071825距離0.14468426940260257 km
存在メッシュ：50325662232値51.46476049698403距離0.2893685387995886 km
存在メッシュ：50325662213値42.31809816845348距離0.1851577137438622 km
存在メッシュ：50325662142値26.066666666666666距離0.0 km
存在メッシュ：50325662232値51.46476049698403距離0.2893685387995886 km
存在メッシュ：50325662231値38.765713582071825距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値39.277816964946766距離0.14468426940260257 km
存在メッシュ：50325662142値

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値17.203318022457815距離0.14468426940260257 km
存在メッシュ：50325662142値11.133333333333333距離0.0 km
存在メッシュ：50325662213値18.901313170665393距離0.1851577137438622 km
存在メッシュ：50325662231値17.203318022457815距離0.14468426940260257 km
存在メッシュ：50325662142値11.133333333333333距離0.0 km
存在メッシュ：50325662232値23.273302711346663距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値18.901313170665393距離0.1851577137438622 km
存在メッシュ：50325662231値17.203318022457815距離0.14468426940260257 km
存在メッシュ：50325662232値23.273302711346663距離0.2893685387995886 km
存在メッシュ：50325662213値18.901313170665393距離0.1851577137438622 km
存在メッシュ：50325662142値11.133333333333333距離0.0 km
存在メッシュ：50325662232値23.273302711346663距離0.2893685387995886 km
存在メッシュ：50325662231値17.203318022457815距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値15.797468597437073距離0.14468426940260257 km
存在メッシュ：50325662142値10.8距離0.0 km
存在メッシュ：503256622

In [156]:
connection.close()

In [111]:
#CreateGeojsonFile